In [7]:
import pandas as pd
import openpyxl
import json
import requests
import matplotlib.pyplot as plt

#### Read ticker symbols of stocks from etoro sheet (exported to another document)

In [10]:
# tickers = investment_history[investment_history['Asset type'] == 'Stocks']['Details'].str.split('/',expand=True)[0].unique()

from tickerlist import tickers
tickers

['UAL',
 'AF.PA',
 'T',
 'JNJ',
 'DIS',
 'AAPL',
 'TSLA',
 'JPM',
 'GOOG',
 'AMD',
 'INGA.NV',
 'SHEL.L',
 'SMSN.L',
 'WMT',
 'NVDA',
 'WBD',
 'MMM',
 'AMZN',
 'COST']

#### Read in json files (requests not made repetitively)

In [14]:
def read_json(ticker_list, statement):

    with open(f'D:\lianz\Desktop\Python\data_science_discovery\personal_finance\{statement}/{ticker_list}.json') as json_file:
        x = json.load(json_file)

    return x

In [15]:
company_statements = ['income-statement',
                      'cash-flow-statement', 'balance-sheet-statement']

In [16]:
d = read_json(tickers[0],company_statements[0])

In [17]:
[d[i]['calendarYear'] for i in range(len(d))]

['2021', '2020', '2019', '2018', '2017']

In [18]:
d[0].items()

dict_items([('date', '2021-12-31'), ('symbol', 'UAL'), ('reportedCurrency', 'USD'), ('cik', '0000100517'), ('fillingDate', '2022-02-18'), ('acceptedDate', '2022-02-18 16:16:13'), ('calendarYear', '2021'), ('period', 'FY'), ('revenue', 24634000000), ('costOfRevenue', 23913000000), ('grossProfit', 721000000), ('grossProfitRatio', 0.02926849070390517), ('researchAndDevelopmentExpenses', 0.0), ('generalAndAdministrativeExpenses', 0.0), ('sellingAndMarketingExpenses', 677000000), ('sellingGeneralAndAdministrativeExpenses', 677000000), ('otherExpenses', 412000000), ('operatingExpenses', 1089000000), ('costAndExpenses', 25002000000), ('interestIncome', 36000000), ('interestExpense', 1577000000), ('depreciationAndAmortization', 2485000000), ('ebitda', 1505000000), ('ebitdaratio', 0.06109442234310303), ('operatingIncome', -1022000000), ('operatingIncomeRatio', -0.04148737517252578), ('totalOtherIncomeExpensesNet', -1535000000), ('incomeBeforeTax', -2557000000), ('incomeBeforeTaxRatio', -0.10379

In [59]:
l = []
dict_holder = {}

for n in ['epsdiluted','netIncome','revenue']:
    # print(n)

    for i,x in enumerate(d[::-1]):
        dict_holder[x['calendarYear']] = x[f'{n}']
    l.append(dict_holder)
    dict_holder = {}
l

[{'2017': 7.02, '2018': 7.7, '2019': 11.58, '2020': -25.3, '2021': -6.1},
 {'2017': 2131000000,
  '2018': 2129000000,
  '2019': 3009000000,
  '2020': -7069000000,
  '2021': -1964000000},
 {'2017': 37736000000,
  '2018': 41303000000,
  '2019': 43259000000,
  '2020': 15355000000,
  '2021': 24634000000}]

In [105]:
eps_df = pd.DataFrame.from_records(l, index=['epsdiluted','netIncome','revenue']).T
eps_df

,epsdiluted,netIncome,revenue
2017,7.02,2.131000e+09,3.773600e+10
2018,7.70,2.129000e+09,4.130300e+10
2019,11.58,3.009000e+09,4.325900e+10
2020,-25.30,-7.069000e+09,1.535500e+10
2021,-6.10,-1.964000e+09,2.463400e+10


In [81]:
terms_interested = {'revenue': 'revenue',
                    'gross margin%': 'grossProfitRatio',
                    'Operating Income': 'operatingIncome',
                    'Operating Margin %': 'operatingIncomeRatio',
                    'Net Income': 'netIncome',
                    'Net Income Margin': 'netIncomeRatio',
                    'Earnings per Share': 'epsdiluted',
                    'Shares Oustanding (diluted)': 'weightedAverageShsOutDil'}

In [84]:
items_interested = terms_interested.values()

for items in items_interested:
    print(items)

revenue
grossProfitRatio
operatingIncome
operatingIncomeRatio
netIncome
netIncomeRatio
epsdiluted
weightedAverageShsOutDil


In [83]:
[items for items in items_interested if items in d[0].keys()]

['revenue',
 'grossProfitRatio',
 'operatingIncome',
 'operatingIncomeRatio',
 'netIncome',
 'netIncomeRatio',
 'epsdiluted',
 'weightedAverageShsOutDil']

In [103]:
arrangement = (2,3)
column_names = []

for i,nrow in enumerate(arrangement):
    for ncolumn in range(1,nrow+1):
        column_names.append(f'c_{i}_{ncolumn}')

column_names

['c_0_1', 'c_0_2', 'c_1_1', 'c_1_2', 'c_1_3']

In [108]:
eps_df.index

Index(['2017', '2018', '2019', '2020', '2021'], dtype='object')

In [116]:
eps_df


,epsdiluted,netIncome,revenue
2017,7.02,2.131000e+09,3.773600e+10
2018,7.70,2.129000e+09,4.130300e+10
2019,11.58,3.009000e+09,4.325900e+10
2020,-25.30,-7.069000e+09,1.535500e+10
2021,-6.10,-1.964000e+09,2.463400e+10


In [126]:
pd.options.display.float_format = '{:.2%}'.format
eps_df['epsdiluted'].pct_change(periods=1).fillna(0)


2017        NaN
2018      9.69%
2019     50.39%
2020   -318.48%
2021    -75.89%
Name: epsdiluted, dtype: float64